In [ ]:
# https://kaggleglm.s3-us-west-2.amazonaws.com/chem/l1.zip

In [498]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
import gc
from fastai.core import parallel

In [561]:
subs = [
    'loss-5.7552val-2.9950', 'loss-4.9516val-3.0042', 'loss-5.9943val-2.7766',
    'loss-4.0414val-2.7287', 'loss-4.9516val-2.8229', 'loss-5.6540val-3.0131',
    'loss-5.1336val-3.0759', 'loss-5.5435val-3.0776', 'loss-3.5933val-3.0637',
    'loss-5.6477val-3.0793', 'loss-4.7210val-3.0501', 'loss-6.8039val-2.8016',
    'loss-5.8718val-3.0917', 'loss-5.3812val-3.0901',
    'val-2.3925_lmae0-2.1331', 'loss-5.1367val-3.0945',
    'loss-6.1966val-3.0929', 'loss-0.5834val-3.0490_lmae2-2.1933'
]

In [562]:
train_x = None
for sub in subs:
    df = pd.read_csv('%s_val' % sub)
    df = df.drop(['id'], axis=1)
    if train_x is None:
        train_x = df
        train_x['scalar_coupling_constant_%s' %
                sub] = df['scalar_coupling_constant']
    else:
        train_x['scalar_coupling_constant_%s' %
                sub] = df['scalar_coupling_constant']
train_x = train_x.drop(['scalar_coupling_constant'], axis=1).astype('float32')
train_x.head()

,scalar_coupling_constant_loss-5.7552val-2.9950,scalar_coupling_constant_loss-4.9516val-3.0042,scalar_coupling_constant_loss-5.9943val-2.7766,scalar_coupling_constant_loss-4.0414val-2.7287,scalar_coupling_constant_loss-4.9516val-2.8229,scalar_coupling_constant_loss-5.6540val-3.0131,scalar_coupling_constant_loss-5.1336val-3.0759,scalar_coupling_constant_loss-5.5435val-3.0776,scalar_coupling_constant_loss-3.5933val-3.0637,scalar_coupling_constant_loss-5.6477val-3.0793,scalar_coupling_constant_loss-4.7210val-3.0501,scalar_coupling_constant_loss-6.8039val-2.8016,scalar_coupling_constant_loss-5.8718val-3.0917,scalar_coupling_constant_loss-5.3812val-3.0901,scalar_coupling_constant_val-2.3925_lmae0-2.1331,scalar_coupling_constant_loss-5.1367val-3.0945,scalar_coupling_constant_loss-6.1966val-3.0929,scalar_coupling_constant_loss-0.5834val-3.0490_lmae2-2.1933
0,83.503128,83.583549,83.352898,83.300529,83.407120,83.482079,83.496063,83.465378,83.403992,83.446213,83.573364,82.900406,83.455582,83.568161,83.462463,83.558289,83.454376,0.0
1,-2.245183,-2.236212,-2.195633,-2.275180,-1.826861,-2.221239,-2.180604,-2.165577,-2.258729,-2.195435,-2.204502,-2.177305,-2.173209,-2.212488,0.000000,-2.214615,-2.177195,0.0
2,-11.567842,-11.562995,-11.576006,-11.587559,-11.596402,-11.558853,-11.559732,-11.567150,-11.585417,-11.565225,-11.549726,-11.544003,-11.567085,-11.552212,0.000000,-11.554241,-11.567059,0.0
3,-11.566397,-11.564421,-11.565042,-11.591578,-11.590795,-11.560163,-11.568606,-11.573649,-11.589661,-11.565928,-11.549247,-11.544028,-11.570606,-11.553642,0.000000,-11.555588,-11.570747,0.0
4,3.245485,3.295776,3.181636,3.254843,3.289506,3.263389,3.243291,3.244716,3.258291,3.251238,3.237890,3.290614,3.250960,3.238988,0.000000,3.240456,3.250406,0.0


In [563]:
#corr = train_x.corr(method='spearman')
#corr

In [564]:
if False:
    seen = dict()
    data = []
    for col1 in corr.columns:
        for col2 in corr.columns:
            if col1 != col2 and col1 + col2 not in seen.keys(
            ) and col2 + col1 not in seen.keys():
                seen[col1 + col2] = 1
                seen[col2 + col1] = 1
                data.append((col1, col2, corr[col1][col2]))
    data
    sorted(data, key=lambda x: x[2], reverse=False)

In [565]:
test_x = None
for sub in subs:
    df = pd.read_csv('%s' % sub)
    df = df.drop(['id'], axis=1)
    if test_x is None:
        test_x = df
        test_x['scalar_coupling_constant_%s' %
               sub] = df['scalar_coupling_constant']
    else:
        test_x['scalar_coupling_constant_%s' %
               sub] = df['scalar_coupling_constant']
test_x = test_x.drop(['scalar_coupling_constant'], axis=1).astype('float32')
test_x.head()

,scalar_coupling_constant_loss-5.7552val-2.9950,scalar_coupling_constant_loss-4.9516val-3.0042,scalar_coupling_constant_loss-5.9943val-2.7766,scalar_coupling_constant_loss-4.0414val-2.7287,scalar_coupling_constant_loss-4.9516val-2.8229,scalar_coupling_constant_loss-5.6540val-3.0131,scalar_coupling_constant_loss-5.1336val-3.0759,scalar_coupling_constant_loss-5.5435val-3.0776,scalar_coupling_constant_loss-3.5933val-3.0637,scalar_coupling_constant_loss-5.6477val-3.0793,scalar_coupling_constant_loss-4.7210val-3.0501,scalar_coupling_constant_loss-6.8039val-2.8016,scalar_coupling_constant_loss-5.8718val-3.0917,scalar_coupling_constant_loss-5.3812val-3.0901,scalar_coupling_constant_val-2.3925_lmae0-2.1331,scalar_coupling_constant_loss-5.1367val-3.0945,scalar_coupling_constant_loss-6.1966val-3.0929,scalar_coupling_constant_loss-0.5834val-3.0490_lmae2-2.1933
0,15.684770,15.999567,8.700197,14.389465,15.555782,15.390451,16.029009,20.745644,17.874069,19.935101,27.552090,17.692743,26.445042,16.503752,0.000000,16.665428,26.011997,0.0
1,188.352219,193.554047,187.950882,154.279251,192.626099,191.664368,193.443756,195.215378,199.125443,190.596954,191.499603,196.137146,191.219727,194.059738,198.874146,194.164871,191.133850,0.0
2,6.846854,7.092568,5.505550,5.256055,6.581933,6.141023,5.023911,4.923062,3.728927,4.947999,5.427936,5.326140,4.832309,5.151481,0.000000,5.157649,4.815150,0.0
3,188.521500,193.675003,187.018097,161.308258,193.716049,191.680237,193.489624,195.263107,199.084839,191.099167,191.931992,196.120132,191.482742,194.440063,198.831512,194.527435,191.400040,0.0
4,15.910284,16.058594,8.810644,17.046246,15.665305,15.159359,15.948315,19.442461,17.889143,19.411510,25.838770,17.708519,24.855202,16.336029,0.000000,16.460855,24.449944,0.0


In [605]:
corr = test_x.corr(method='spearman')
corr

,scalar_coupling_constant_loss-5.7552val-2.9950,scalar_coupling_constant_loss-4.9516val-3.0042,scalar_coupling_constant_loss-5.9943val-2.7766,scalar_coupling_constant_loss-4.0414val-2.7287,scalar_coupling_constant_loss-4.9516val-2.8229,scalar_coupling_constant_loss-5.6540val-3.0131,scalar_coupling_constant_loss-5.1336val-3.0759,scalar_coupling_constant_loss-5.5435val-3.0776,scalar_coupling_constant_loss-3.5933val-3.0637,scalar_coupling_constant_loss-5.6477val-3.0793,scalar_coupling_constant_loss-4.7210val-3.0501,scalar_coupling_constant_loss-6.8039val-2.8016,scalar_coupling_constant_loss-5.8718val-3.0917,scalar_coupling_constant_loss-5.3812val-3.0901,scalar_coupling_constant_val-2.3925_lmae0-2.1331,scalar_coupling_constant_loss-5.1367val-3.0945,scalar_coupling_constant_loss-6.1966val-3.0929,scalar_coupling_constant_loss-0.5834val-3.0490_lmae2-2.1933
scalar_coupling_constant_loss-5.7552val-2.9950,1.000000,0.999980,0.999869,0.999781,0.999835,0.999979,0.999966,0.999962,0.999952,0.999956,0.999949,0.999879,0.999960,0.999957,0.624498,0.999958,0.999960,0.117401
scalar_coupling_constant_loss-4.9516val-3.0042,0.999980,1.000000,0.999866,0.999779,0.999833,0.999988,0.999969,0.999965,0.999954,0.999957,0.999950,0.999876,0.999962,0.999958,0.624498,0.999959,0.999962,0.117401
scalar_coupling_constant_loss-5.9943val-2.7766,0.999869,0.999866,1.000000,0.999761,0.999809,0.999868,0.999869,0.999871,0.999872,0.999872,0.999866,0.999857,0.999874,0.999870,0.624497,0.999871,0.999874,0.117401
scalar_coupling_constant_loss-4.0414val-2.7287,0.999781,0.999779,0.999761,1.000000,0.999749,0.999779,0.999780,0.999779,0.999787,0.999779,0.999780,0.999755,0.999781,0.999782,0.624496,0.999783,0.999781,0.117401
scalar_coupling_constant_loss-4.9516val-2.8229,0.999835,0.999833,0.999809,0.999749,1.000000,0.999835,0.999836,0.999836,0.999841,0.999836,0.999834,0.999812,0.999838,0.999837,0.624497,0.999838,0.999838,0.117401
scalar_coupling_constant_loss-5.6540val-3.0131,0.999979,0.999988,0.999868,0.999779,0.999835,1.000000,0.999971,0.999972,0.999955,0.999961,0.999953,0.999880,0.999968,0.999963,0.624498,0.999964,0.999968,0.117401
scalar_coupling_constant_loss-5.1336val-3.0759,0.999966,0.999969,0.999869,0.999780,0.999836,0.999971,1.000000,0.999987,0.999973,0.999976,0.999967,0.999878,0.999981,0.999976,0.624498,0.999977,0.999981,0.117400
scalar_coupling_constant_loss-5.5435val-3.0776,0.999962,0.999965,0.999871,0.999779,0.999836,0.999972,0.999987,1.000000,0.999968,0.999982,0.999971,0.999883,0.999994,0.999984,0.624498,0.999985,0.999993,0.117401
scalar_coupling_constant_loss-3.5933val-3.0637,0.999952,0.999954,0.999872,0.999787,0.999841,0.999955,0.999973,0.999968,1.000000,0.999963,0.999955,0.999880,0.999966,0.999963,0.624499,0.999964,0.999966,0.117400
scalar_coupling_constant_loss-5.6477val-3.0793,0.999956,0.999957,0.999872,0.999779,0.999836,0.999961,0.999976,0.999982,0.999963,1.000000,0.999986,0.999884,0.999987,0.999980,0.624498,0.999981,0.999987,0.117401


In [566]:
test_y = pd.read_csv('temp_-3.225.csv')
test_y = test_y.drop(['id'],
                     axis=1)['scalar_coupling_constant'].astype('float32')
test_y.head()

0     19.649082
1    191.267838
2      5.250949
3    191.589539
4     19.775169
Name: scalar_coupling_constant, dtype: float32

In [567]:
type_index = {
    '1JHC': 0,
    '2JHH': 1,
    '1JHN': 2,
    '2JHN': 3,
    '2JHC': 4,
    '3JHH': 5,
    '3JHC': 6,
    '3JHN': 7
}
test_types = pd.read_csv('test.csv')['type']
test_types_idx = []
for value in test_types.values:
    test_types_idx.append(type_index[value])
test_types_idx = np.array(test_types_idx)
test_types_idx

array([4, 0, 5, 0, ..., 4, 6, 4, 0])

In [568]:
train_y = pd.read_csv('validation_targets')
train_y = train_y.drop(['id'],
                       axis=1)['scalar_coupling_constant'].astype('float32')

In [569]:
types = pd.read_csv('validation_types')
types = types.drop(['id'], axis=1)['type']

In [570]:
from sklearn.linear_model import *

In [601]:
from sklearn.ensemble import VotingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.experimental import enable_hist_gradient_boosting  # noqa

reg1 = KNeighborsRegressor(n_neighbors=20)  # -3.19
reg2 = LinearRegression()  # -3.16
reg3 = LinearRegression(fit_intercept=False)
reg4 = RANSACRegressor()  #-3.177
reg5 = TheilSenRegressor()  #-3.177
#reg6 = RandomForestRegressor(n_estimators=10) #-4.04
#reg6 = BayesianRidge() # -3.16867
reg6 = HuberRegressor()  # -3.18393.18761196871987

weights = np.array([
    3.1815764009952545, 3.13541778922081, 3.159316837787628, 3.19497118716738
]) - 3.135
#weights = np.array([3.186998337507248, 3.1445012986660004, 3.1850613355636597, 3.206811179044068]) - 3.14
#weights = np.array([3.1878311038017273, 3.1436126828193665, 3.1846345961093903, 3.20678490653821]) - 3.14
#weights = np.array([3.1885430812835693, 3.1473455131053925, 3.1755231618881226, 3.209466837133884]) - 3.14
weights = weights / weights.sum()

print(weights, weights.sum())
#weights=None
model = VotingRegressor(estimators=[('knr', reg1), ('lr', reg2),
                                    ('ransac', reg4), ('th', reg5)],
                        weights=weights)
#model = VotingRegressor(estimators=[
#    ('knr', reg1), ('lr2', reg2), ('ransac', reg4), ('th', reg5), ('hb', reg6)],n_jobs=-1)
#model = VotingRegressor(estimators=[('hb', reg6)])
#model=reg6

#from sklearn.neural_network import MLPRegressor
#from mlxtend.regressor import StackingRegressor
#model = StackingRegressor(regressors=[reg1, reg2, reg4, reg5],
#                           meta_regressor=MLPRegressor())

#model=reg5

[0.354781 0.003182 0.185226 0.456811] 1.0


In [572]:
#[0.340928 0.035812 0.193789 0.42947 ] 1.0

In [579]:
y = train_y
X = train_x

In [589]:
def eval_lmae(gt, pred, types):
    types = types.values.astype(np.int)
    gt = np.asarray(gt)
    pred = np.asarray(pred)
    loss = 0.
    for type in np.unique(types):
        mask = types == type
        loss += np.log(np.mean(np.abs(gt[mask] - pred[mask])))
    loss = loss / len(np.unique(types))
    return loss

In [602]:
# Run CV
# https://www.kaggle.com/aharless/simple-catboost-cv-lb-281
OPTIMIZE_ROUNDS = False
y_test_pred = None
y_test_pred = [None, None, None, None, None, None, None, None]

K = 1


def calc_type(type, iii):
    loss = 0
    y_test_pred = None
    for i, (train_index, test_index) in enumerate([(range(len(X)),
                                                    range(len(y)))]):
        gc.collect()

        # Create data for this fold
        y_train, y_valid = y.iloc[train_index], y.iloc[test_index]
        X_train, X_valid = X.iloc[train_index, :], X.iloc[test_index, :]
        type_train, type_valid = types.iloc[train_index], types.iloc[
            test_index]

        y_train, y_valid, X_train, X_valid, type_train, type_valid = y_train[
            type_train == type], y_valid[type_valid == type], X_train[
                type_train == type], X_valid[type_valid == type], type_train[
                    type_train == type], type_valid[type_valid == type]

        fit_model = model.fit(X_train, y_train)

        # Generate validation predictions for this fold
        pred = fit_model.predict(X_valid)
        loss += eval_lmae(y_valid, pred, type_valid)

        # Accumulate test set predictions
        if y_test_pred is None:
            y_test_pred = fit_model.predict(test_x)
        else:
            y_test_pred += fit_model.predict(test_x)
        return (type, y_test_pred, loss)

In [603]:
res = parallel(calc_type, range(8))

In [604]:
y_test_pred, loss = [None] * 8, [None] * 8
for (type, y_test_preds, type_loss) in res:
    y_test_pred[type] = y_test_preds
    loss[type] = type_loss
for i in range(8):
    print(f"Type {i} LMAE: {loss[i]}")
print(f"\nFinal  LMAE: {np.mean(loss)}")

Type 0 LMAE: -2.2191930191385847
Type 1 LMAE: -3.665479538900041
Type 2 LMAE: -2.3310372059898725
Type 3 LMAE: -3.580872211802855
Type 4 LMAE: -3.2571294916178863
Type 5 LMAE: -3.775351639613008
Type 6 LMAE: -3.16076846719372
Type 7 LMAE: -3.769642456869231

Final  LMAE: -3.21993425389065


In [537]:
y_test_pred_list = y_test_pred
y_test_pred = []

for idx, type_idx in enumerate(test_types_idx):
    y_test_pred.append(y_test_pred_list[type_idx][idx])
y_test_pred = np.array(y_test_pred)

In [538]:
y_test_pred

array([ 18.203103, 192.120042,   5.186197, 192.585937, ...,   4.219772,   1.878515,   4.046821, 120.114081])

In [539]:
df = pd.read_csv('%s' % subs[0])
df.head()

,id,scalar_coupling_constant
0,4658147,15.684770
1,4658148,188.352219
2,4658149,6.846854
3,4658150,188.521500
4,4658151,15.910284


In [540]:
df['scalar_coupling_constant'] = y_test_pred

In [541]:
df.head()

,id,scalar_coupling_constant
0,4658147,18.203103
1,4658148,192.120042
2,4658149,5.186197
3,4658150,192.585937
4,4658151,17.986942


In [542]:
sub_fname = 'LMAE_' + str(
    np.array(loss).mean()) + '-LinearRegressionByType-1fold'
sub_fname

'LMAE_-3.220141338687831-LinearRegressionByType-1fold'

In [543]:
df.to_csv(sub_fname, index=False)

In [521]:
comp = 'champs-scalar-coupling'
m = ' '.join(subs) + ' estimators: ' + ' '.join(
    [name for name, _ in model.estimators])
m

'loss-5.7552val-2.9950 loss-4.9516val-3.0042 loss-5.9943val-2.7766 loss-4.0414val-2.7287 loss-4.9516val-2.8229 loss-5.6540val-3.0131 loss-5.1336val-3.0759 loss-5.5435val-3.0776 loss-3.5933val-3.0637 loss-5.6477val-3.0793 loss-4.7210val-3.0501 loss-6.8039val-2.8016 loss-5.8718val-3.0917 loss-5.3812val-3.0901 val-2.3925_lmae0-2.1331 loss-5.1367val-3.0945 loss-6.1966val-3.0929 estimators: knr lr ransac th'

In [560]:
sub_fname

'LMAE_-3.220141338687831-LinearRegressionByType-1fold'

In [149]:
!kaggle competitions submit -c {comp} -f {sub_fname} -m ' '

100%|██████████████████████████████████████| 64.1M/64.1M [00:04<00:00, 15.1MB/s]
Successfully submitted to Predicting Molecular Properties